this tutorial is based on ``daner_PGC_SCZ52_0513a.hq2.gz`` file that can be downloaded from
https://pgc.unc.edu/for-researchers/download-results/

(Note that PGC released new SCZ GWAS in 2022, for latest and largest GWAS on schizophrenia use PGC3_SCZ_wave3.european.autosome.public.v3.vcf.tsv.gz)

In [1]:
import pandas as pd
df=pd.read_csv('sumstats/daner_PGC_SCZ52_0513a.hq2.gz', delim_whitespace=True)
df.head()

,CHR,SNP,BP,A1,A2,FRQ_A_35476,FRQ_U_46839,INFO,OR,SE,P,ngt,Direction,HetISqt,HetChiSq,HetDf,HetPVa
0,8,rs62513865,101592213,T,C,0.0767,0.08020,0.938,0.97922,0.0208,0.3136,0,--++-----++--------++++?+?+-+-+---++----+----?...,0.0,46.604,48,0.5301
1,8,rs79643588,106973048,A,G,0.0940,0.09850,1.000,0.98472,0.0184,0.4039,0,-+---+-----+----+++--+-?+?+-----++++++++-++--?...,-2.3,48.886,48,0.4373
2,8,rs17396518,108690829,T,G,0.5500,0.54500,0.977,1.00461,0.0108,0.6690,13,--+-+--+--++++-++--+++++++--+-++++-++++-+++---...,13.9,61.527,51,0.1484
3,8,rs138449472,108580746,A,G,0.0085,0.00808,0.742,1.07519,0.0753,0.3356,0,++?-+?-+-?-+++++?----?+?+?+---+---+++-?-?--+??...,0.0,29.905,38,0.8228
4,8,rs983166,108681675,A,C,0.5570,0.55400,0.998,1.00491,0.0106,0.6418,0,--+-+-----++--+++++++++--+-+-++++--+-++-++++--...,0.0,44.898,51,0.7134


In [2]:
# find how many SNPs are in GWAS
len(df)

10172956

In [3]:
# rename columns if needed; drop colums we're not interested in
df.rename(columns={'P':'PVALUE'}, inplace=True)
df.drop(columns=['ngt', 'FRQ_A_35476', 'FRQ_U_46839', 'Direction', 'HetISqt', 'HetChiSq', 'HetDf', 'HetPVa'], inplace=True)
df.head()

,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE
0,8,rs62513865,101592213,T,C,0.938,0.97922,0.0208,0.3136
1,8,rs79643588,106973048,A,G,1.000,0.98472,0.0184,0.4039
2,8,rs17396518,108690829,T,G,0.977,1.00461,0.0108,0.6690
3,8,rs138449472,108580746,A,G,0.742,1.07519,0.0753,0.3356
4,8,rs983166,108681675,A,C,0.998,1.00491,0.0106,0.6418


In [16]:
# add sample size (the numbers here are taken from FRQ_A_35476 and FRQ_U_46839 columns)
df['N'] = 35476+46839
df['Neff'] = 4/(1/35476 + 1/46839)

import numpy as np
from scipy import stats
df['Z'] = -stats.norm.ppf(df['PVALUE'].values*0.5)*np.sign(np.log(df['OR'].values))
df.head()

,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE,N,Z,Neff
0,8,rs62513865,101592213,T,C,0.938,0.97922,0.0208,0.3136,82315,-1.007697,80746.418709
1,8,rs79643588,106973048,A,G,1.000,0.98472,0.0184,0.4039,82315,-0.834676,80746.418709
2,8,rs17396518,108690829,T,G,0.977,1.00461,0.0108,0.6690,82315,0.427521,80746.418709
3,8,rs138449472,108580746,A,G,0.742,1.07519,0.0753,0.3356,82315,0.962895,80746.418709
4,8,rs983166,108681675,A,C,0.998,1.00491,0.0106,0.6418,82315,0.465184,80746.418709


In [20]:
# make sure 'CHR' is an integer column; check types for other columns
df = df[~df['CHR'].isnull()].copy()
df['CHR']=df['CHR'].astype(int)
#df.to_csv('PGC_SCZ_2014.csv', index=False, sep='\t')
print(df.dtypes)
df.head()


CHR         int64
SNP        object
BP          int64
A1         object
A2         object
INFO      float64
OR        float64
SE        float64
PVALUE    float64
N           int64
Z         float64
Neff      float64
dtype: object


,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE,N,Z,Neff
0,8,rs62513865,101592213,T,C,0.938,0.97922,0.0208,0.3136,82315,-1.007697,80746.418709
1,8,rs79643588,106973048,A,G,1.000,0.98472,0.0184,0.4039,82315,-0.834676,80746.418709
2,8,rs17396518,108690829,T,G,0.977,1.00461,0.0108,0.6690,82315,0.427521,80746.418709
3,8,rs138449472,108580746,A,G,0.742,1.07519,0.0753,0.3356,82315,0.962895,80746.418709
4,8,rs983166,108681675,A,C,0.998,1.00491,0.0106,0.6418,82315,0.465184,80746.418709


In [15]:
# remaining problem - SNPs without an rs#
# (this can be solved with cleansumstats pipeline - https://github.com/BioPsyk/cleansumstats )
df[df['A1'].isin(['A', 'T', 'C', 'G']) & ~df['SNP'].str.startswith('rs')].head()

,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE,N,Z
2457,8,chr8_101727716,101727716,T,C,0.494,1.02932,0.0365,0.4286,82315,0.791590
3644,8,chr8_103573006,103573006,A,G,0.959,0.99890,0.0112,0.9206,82315,-0.099678
13064,8,chr8_103573015,103573015,A,G,0.961,1.00471,0.0123,0.7006,82315,0.384511
23627,8,chr8_101721756,101721756,A,C,0.511,0.99471,0.0304,0.8622,82315,-0.173574
24043,8,chr8_103573026,103573026,A,C,0.709,0.95705,0.0558,0.4314,82315,-0.786798


In [11]:
# remaining problem - non-ACTG SNPs
# (this is something cleansumstats pipeline won't solve)
df[~df['A1'].isin(['A', 'T', 'C', 'G'])].head()

,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE,N,Z
7,8,chr8_103128181_I,103128181,D,I3,0.996,1.01501,0.0147,0.312500,82315,1.009990
10,8,chr8_104098842_D,104098842,I5,D,0.977,0.99392,0.0177,0.731000,82315,-0.343796
13,8,chr8_106614944_I,106614944,I2,D,0.998,1.02480,0.0127,0.054320,82315,1.924276
21,8,chr8_100846770_I,100846770,I2,D,0.604,1.06876,0.0677,0.325700,82315,0.982812
22,8,chr8_109768961_D,109768961,I5,D,0.861,0.96377,0.0128,0.004049,82315,-2.874319


In [22]:
# ambiguous SNPs - A/T and C/G SNPs
df[(df['A1']+df['A2']).isin(['AT', 'TA', 'CG', 'GC'])].head()

,CHR,SNP,BP,A1,A2,INFO,OR,SE,PVALUE,N,Z,Neff
6,8,rs7014597,104152280,C,G,0.995,0.98639,0.0147,0.3496,82315,-0.935365,80746.418709
11,8,rs72670434,108166508,A,T,0.988,1.00361,0.0111,0.7467,82315,0.322994,80746.418709
23,8,rs34397009,106490696,C,G,0.990,0.99661,0.0164,0.8349,82315,-0.208421,80746.418709
42,8,rs28678866,108828542,A,T,0.632,1.07358,0.0459,0.1217,82315,1.547677,80746.418709
46,8,rs72670454,108204493,C,G,0.708,1.00874,0.0791,0.9125,82315,0.109886,80746.418709
